In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import joblib
import scipy

some duplicates removal (ignore and dont use)

---

In [79]:
d_answer_emb = np.load("../data/features/answer_bge_m3.npz")
d_question_emb = np.load("../data/features/question_bge_m3.npz")
d_title_emb = np.load("../data/features/title_bge_m3.npz")

In [ ]:
answer_emb = d_answer_emb['features']
answer_ids = d_answer_emb['ids']

question_emb = d_question_emb['features']
question_ids = d_question_emb['ids']

title_emb = d_title_emb['features']
title_ids = d_title_emb['ids']

In [82]:
duplicate_idx = np.load("../data/processed/duplicate_idx.npy")  

In [83]:
len(duplicate_idx)

218607

In [84]:
mask = ~np.isin(answer_ids, duplicate_idx)

In [87]:
answer_emb_clean = answer_emb[mask]
question_emb_clean = question_emb[mask]
title_emb_clean = title_emb[mask]

new_idx = np.arange(len(answer_emb_clean))

In [90]:
np.savez_compressed("../data/features/answer_bge_m3_d.npz", features=answer_emb_clean, ids=new_idx)
np.savez_compressed("../data/features/question_bge_m3_d.npz", features=question_emb_clean, ids=new_idx)
np.savez_compressed("../data/features/title_bge_m3_d.npz", features=title_emb_clean, ids=new_idx)

---

In [19]:
df = pd.read_csv("../data/targets/targets.csv")

In [25]:
for col in df.columns:
    print(f"Column: {col}, Unique values: {df[col].nunique()}")

Column: SUBCATEGORY_CLEAN_enc, Unique values: 15
Column: PRIORITY_enc, Unique values: 2
Column: AVARIYA_enc, Unique values: 2


In [38]:
import joblib
le = joblib.load("../models/vectorizers/label_encoder_AVARIYA.pkl")

In [44]:
le = joblib.load("../models/vectorizers/label_encoder_SUBCATEGORY_CLEAN.pkl")
for c in le.classes_:
    print(f"Class: {c}, Index: {le.transform([c])[0]}")

Class: Авария, Index: 0
Class: Доступ к ИСОД, Index: 1
Class: Запрос на администрирование, Index: 2
Class: Запрос на доработку, Index: 3
Class: Запрос статуса, Index: 4
Class: Консультация, Index: 5
Class: Коррекция данных, Index: 6
Class: Настройка ПО и оборудования, Index: 7
Class: ОШС, Index: 8
Class: Оборудование, Index: 9
Class: Программное обеспечение, Index: 10
Class: Программное обеспечение. Региональные ПТК, Index: 11
Class: Прочее, Index: 12
Class: Прочие вопросы, Index: 13
Class: СПГУ. Несколько модулей, Index: 14


---

In [186]:
df = pd.read_csv("../data/processed/cleaned_dataset.csv")
df.columns

Index(['S_NAME', 'QUESTION', 'ANSWER', 'OPEN_TIME_', 'RESOLVE_TIME_',
       'CLOSE_TIME_', 'ATC_NEXT_BREACH_', 'TITLE', 'SUBCATEGORY', 'REG',
       'PRIORITY', 'AVARIYA', 'SOURCE', 'QUESTION_FULL', 'SUBCATEGORY_CLEAN'],
      dtype='object')

### Сплит данных

In [109]:
# Будем стратифицировать по всем таргетам сразу
df['stratify_key'] = df['SUBCATEGORY_CLEAN'].astype(str) + '_' + df['PRIORITY'].astype(str) + df['AVARIYA'].astype(str)
counts = df['stratify_key'].value_counts()
rare_classes = counts[counts < 20].index
df['stratify_key'] = df['stratify_key'].replace(rare_classes, 'OTHER')
df['stratify_key'].value_counts()

all_ids = df.index.values
train_val_ids, test_ids = train_test_split(
    all_ids,
    test_size=0.10,
    random_state=42,
    stratify=df['stratify_key'],
    shuffle=True
)

# Subset stratify_key to the train_val_ids
train_val_stratify = df.loc[train_val_ids, 'stratify_key']

train_ids, val_ids = train_test_split(
    train_val_ids,
    test_size=0.15,
    random_state=42,
    stratify=train_val_stratify,
    shuffle=True
)

np.save("../data/splits/train.npy", train_ids)
np.save("../data/splits/val.npy", val_ids)
np.save("../data/splits/test.npy", test_ids)

Добавим выборку поменьше

In [ ]:
def subsample_stratified(ids, stratify_column, fraction=1/3, random_state=42):
    """Subsample while maintaining stratification"""
    if fraction >= 1.0:
        return ids
    
    # Get the stratify values for these IDs
    stratify_values = stratify_column.loc[ids]
    
    # Perform stratified sampling
    subsampled_ids, _ = train_test_split(
        ids,
        train_size=fraction,
        random_state=random_state,
        stratify=stratify_values,
        shuffle=True
    )
    return subsampled_ids

# # Будем стратифицировать по всем таргетам сразу
# df['stratify_key'] = df['SUBCATEGORY_CLEAN'].astype(str) + '_' + df['PRIORITY'].astype(str) + df['AVARIYA'].astype(str)
# counts = df['stratify_key'].value_counts()
# rare_classes = counts[counts < 20].index
# df['stratify_key'] = df['stratify_key'].replace(rare_classes, 'OTHER')
# df['stratify_key'].value_counts()

# train_ids = np.load('../data/splits/train.npy')
# val_ids = np.load('../data/splits/val.npy')
# test_ids = np.load('../data/splits/test.npy')

# Subsample each split
train_ids_subsampled = subsample_stratified(train_ids, df['stratify_key'], fraction=1/3)
val_ids_subsampled = subsample_stratified(val_ids, df['stratify_key'], fraction=1/3)
test_ids_subsampled = subsample_stratified(test_ids, df['stratify_key'], fraction=1/3)

print(f"Original sizes - Train: {len(train_ids)}, Val: {len(val_ids)}, Test: {len(test_ids)}")
print(f"Subsampled sizes - Train: {len(train_ids_subsampled)}, Val: {len(val_ids_subsampled)}, Test: {len(test_ids_subsampled)}")

np.save("../data/splits/train_small.npy", train_ids_subsampled)
np.save("../data/splits/val_small.npy", val_ids_subsampled)
np.save("../data/splits/test_small.npy", test_ids_subsampled)

Original sizes - Train: 535399, Val: 94483, Test: 69987
Subsampled sizes - Train: 178466, Val: 31494, Test: 23329


In [111]:
df.iloc[train_val_ids]['AVARIYA'].value_counts()

AVARIYA
Нет    619859
Да      10023
Name: count, dtype: int64

## Feature extraction

### Текстовые фичи

In [112]:
# TF-ID
tfidf = TfidfVectorizer(
    max_features=8000,  # можно увеличить для точности
    ngram_range=(1,2),
    min_df=5
)

# Используем train split чтобы не было утечки
tfidf.fit(df.loc[train_ids, 'QUESTION_FULL'])
X_question_full = tfidf.transform(df['QUESTION_FULL'])

Эмбеддинги для всех текстовых фич вычислили отдельно на сервере (bge_m3)

In [ ]:
# def build_sentence_embeddings(df, column, model_path, save_path=None):
#     model = SentenceTransformer(model_path)
#     sentences = df[column].tolist()
#     embeddings = model.encode(sentences, batch_size=64, show_progress_bar=True)
    
#     if save_path:
#         np.savez_compressed(save_path, embeddings)
#     return embeddings

In [ ]:
# build_sentence_embeddings(df, 'QUESTION_FULL', model_path='../models/embeddings/deepvk_USER-bge-m3', save_path='../data/features/question_bge_m3.npz')

### Категориальные фичи

In [53]:
import joblib

le = joblib.load('../models/vectorizers/label_encoder_REG.pkl')

In [184]:
df[cat_cols].nunique()

S_NAME     22
REG       933
SOURCE      5
dtype: int64

In [114]:
ord_encoders = {}

cat_cols = ['S_NAME', 'REG', 'SOURCE']

for col in cat_cols:
    oe = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    oe.fit(df.loc[train_ids, [col]]) 
    df[col + '_enc'] = oe.transform(df[[col]]).flatten()  # Transform and flatten to 1D
    ord_encoders[col] = oe

X_cat = df[[c + '_enc' for c in cat_cols]].values

### Временные фичи

In [115]:
for col in ['OPEN_TIME_', 'RESOLVE_TIME_', 'CLOSE_TIME_', 'ATC_NEXT_BREACH_']:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Encoding cyclical features
hour = df['OPEN_TIME_'].dt.hour
df['hour_sin'] = np.sin(hour * (2. * np.pi / 24.))
df['hour_cos'] = np.cos(hour * (2. * np.pi / 24.))

day_of_week = df['OPEN_TIME_'].dt.dayofweek
df['day_of_week_sin'] = np.sin(day_of_week * (2. * np.pi / 7.))
df['day_of_week_cos'] = np.cos(day_of_week * (2. * np.pi / 7.))

df['is_weekend'] = (day_of_week >= 5).astype(int)

day_of_month = df['OPEN_TIME_'].dt.day
df['day_of_month_sin'] = np.sin(day_of_month * (2. * np.pi / 31.))
df['day_of_month_cos'] = np.cos(day_of_month * (2. * np.pi / 31.))

X_time = df[['day_of_week_sin', 'day_of_week_cos', 'hour_sin', 'hour_cos', 'day_of_month_sin', 'day_of_month_cos', 'is_weekend']].fillna(0).values

### Таргеты

In [116]:
target_cols = ['SUBCATEGORY_CLEAN', 'PRIORITY', 'AVARIYA']
targets = df[target_cols]

label_encoders = {}

for col in target_cols:
    le = LabelEncoder()
    le.fit(df.loc[train_ids, col])
    classes = le.classes_
    df[f'{col}_processed'] = df[col].apply(lambda x: np.where(classes == x)[0][0] if x in classes else -1)
    df[col + '_enc'] = le.transform(df[col])
    label_encoders[col] = le

y = df[[c + '_enc' for c in target_cols]]

### Сохраняем фичи, таргеты и векторизаторы

In [120]:
np.savez_compressed("../data/features/question_tfidf.npz", features=X_question_full, ids=df.index.values)
np.savez_compressed('../data/features/cat_features.npz', features=X_cat, ids=df.index.values)
np.savez_compressed('../data/features/time_features.npz', features=X_time, ids=df.index.values)

y.to_csv('../data/targets/targets.csv', index=False)

joblib.dump(tfidf, '../models/vectorizers/tfidf_vectorizer.pkl')
for col, le in label_encoders.items():
    joblib.dump(le, f'../models/vectorizers/label_encoder_{col}.pkl')
for col, oe in ord_encoders.items():
    joblib.dump(oe, f'../models/vectorizers/ordinal_encoder_{col}.pkl')

In [ ]:
oo = pd.read_csv("../data/targets/targets.csv")

Сохраним текстовые фичи для получения эмбеддингов позднее:

In [128]:
question_fulls = df['QUESTION_FULL']
titles = df['TITLE']
anwers = df['ANSWER']

# question_fulls.to_csv('../torch_container/data/input/question_texts.csv', index=False)
# titles.to_csv('../torch_container/data/input/title_texts.csv', index=False)
anwers.to_csv('../torch_container/data/input/answer_texts.csv', index=False)

Проиндексируем эмбеддинги:

In [46]:
question_bge = np.load('../data/features/question_bge_m3.npy')
title_bge = np.load('../data/features/title_bge_m3.npy')
answer_bge = np.load('../data/features/answer_bge_m3.npy')

In [47]:
np.savez('../data/features/question_bge_m3.npz', features=question_bge, ids=df.index.values)
np.savez('../data/features/title_bge_m3.npz', features=title_bge, ids=df.index.values)
np.savez('../data/features/answer_bge_m3.npz', features=answer_bge, ids=df.index.values)

In [ ]:
timef = np.load('../data/features/time_features.npz')
catf = np.load('../data/features/cat_features.npz')

In [131]:
timef['features'].shape

(699869, 7)